In [367]:
import numpy as np
import numpy.linalg as la
from sklearn.model_selection import train_test_split

In [368]:
# input = np.load("input.npy")
# output = np.load("output.npy")
# x = np.array([input**2, input]).T
# y = output.reshape((output.shape[0], -1))

# tts = train_test_split(x, y)
# train_x = tts[0][0]
# train_y = tts[2][0]
# test_x = tts[1][0]
# test_y = tts[3][0]

# def gen_x(N, d):
#     arr = np.random.randint(100, size=(N,d))
#     return arr

# def gen_y(x):
#     s = np.sum(x, axis=1)
#     boolean = x[:,0] / s
#     return (boolean >= 0.5).astype(np.float32)

def gen_x(N, d):
    arr = np.random.randint(100, size=(N,d))
    return arr

def gen_y(x):
    return (x[:, 0] >= 50).astype(np.float32)

# 2 Neural Networks
## Problem 2.1 Perception

In [373]:
# def cross_entropy_loss(w, x, y_g):
#     # y_g is {-1, 1}
#     return -y_g*np.log(w*x) - (1-y_g)*np.log(1-w*x)

def cross_entropy_loss(y_p, y_g):
    # y_g is {-1, 1}
    return -y_g*np.log(y_p) - (1-y_g)*np.log(1-y_p)

def cross_entropy_deriv(w, x, y_g, predict):
    y_p = predict(w, x)
    return (y_g / y_p) + (1-y_g) / (1-y_p)

def sigmoid(x):
    # assert type(x) == np.ndarray
    denominator = 1 + np.exp(-x)
    return 1 / denominator

def sgn(x):
    return np.sign(x)

def hinge_loss(w, x, y_g, predict):
    y_p = predict(w, x)
    return np.sum(np.maximum(0, 1-y_p*y_g))

def hinge_deriv(w, x, y_g, predict):
    y_p = predict(w, x)
    if y_p*y_g < 1:
        return np.sum(-y_g*x)
    return 0

In [381]:
class Perceptron:
    """Currently only supports a very specific implementation
    """

    def __init__(self, x, y, nlf, ell, dell, loss, lr=0.01, tol=0.01):
        assert (x.shape[0], 1) == (y.shape[0], 1), "Make sure your matrices are (N,d) and (N,1)" # (Class, dim)
        # assert len(x.shape) == 2, "Make sure x is 2D. If not do reshape((N, -1))"
        x = x.reshape((x.shape[0], -1))

        self.nlf_ = nlf # Non linear function (sigmoid)
        self.ell_ = ell
        self.dell_ = dell # Derivative of Loss Function
        self.loss_ = loss
        self.lr_ = lr
        self.tol_ = tol
        self.num_points_ = x.shape[0] # N
        self.dim_ = x.shape[1] + 1 # Add one to dimension for bias, d
        self.w_ = np.random.rand(self.dim_) # Initialize random weights [0, 1)
        self.w_ /= la.norm(self.w_)
        self.x_ = x.copy()# Append a one at beginning for bias
        # np.insert(x.reshape((self.num_points_, -1)), 0, np.ones(self.num_points_), axis=1) 
        self.y_g = y.T.flatten().copy()

    def mini_batch_gradient_descent(self, lr=0.01, batch_size=5, epochs=25):
        error = []
        for _ in range(epochs):
            idx = np.random.randint(self.num_points_, size=batch_size)
            grad = 0
            err = 0
            for i in idx:
                err += self.ell_(self.w_, self.x_[i], self.y_g[i], self.predict)
                grad += self.dell_(self.w_, self.x_[i], self.y_g[i], self.predict)
            grad /= idx.shape[0]
            err /= idx.shape[0]
            error.append((_, err))
            self.w_ = self.w_ - lr * grad
        return self.w_, error
    
    def predict(self, w, x):
        """Performs feed forward with weight summation and non-linear activation
        """
        # assert row.shape[0] == self.dim_, str(row.shape) + " " + str(self.dim_)
        mean = np.average(x)
        std = np.std(x)
        normalized_x = (x - mean) / std
        return self.nlf_(np.sum(w[1:] * normalized_x) + w[0])
        
    def prediction(self, x):
        return self.predict(self.w_, x)

In [382]:
train_x = gen_x(100, 3)
train_y = gen_y(train_x)

In [383]:
# p = Perceptron(train_x, train_y, sigmoid, cross_entropy_loss, cross_entropy_deriv, cross_entropy_loss)
p = Perceptron(train_x, train_y, sgn, hinge_loss, hinge_deriv, cross_entropy_loss)
w, error = p.mini_batch_gradient_descent(batch_size=5, epochs=100)
error

[(0, 0.2),
 (1, 0.2),
 (2, 0.4),
 (3, 0.6),
 (4, 0.4),
 (5, 0.8),
 (6, 0.2),
 (7, 0.8),
 (8, 0.2),
 (9, 0.4),
 (10, 0.6),
 (11, 0.6),
 (12, 0.4),
 (13, 1.0),
 (14, 0.2),
 (15, 0.4),
 (16, 0.4),
 (17, 0.6),
 (18, 0.4),
 (19, 0.4),
 (20, 0.2),
 (21, 0.6),
 (22, 0.2),
 (23, 0.6),
 (24, 0.2),
 (25, 0.2),
 (26, 0.0),
 (27, 0.2),
 (28, 0.2),
 (29, 0.2),
 (30, 0.4),
 (31, 0.6),
 (32, 0.4),
 (33, 0.6),
 (34, 0.6),
 (35, 0.8),
 (36, 0.4),
 (37, 0.6),
 (38, 0.4),
 (39, 0.6),
 (40, 0.0),
 (41, 0.6),
 (42, 0.4),
 (43, 0.6),
 (44, 0.6),
 (45, 0.2),
 (46, 0.6),
 (47, 0.4),
 (48, 0.4),
 (49, 0.2),
 (50, 0.0),
 (51, 0.2),
 (52, 0.6),
 (53, 0.6),
 (54, 0.6),
 (55, 0.6),
 (56, 0.6),
 (57, 0.6),
 (58, 0.4),
 (59, 0.4),
 (60, 0.4),
 (61, 0.4),
 (62, 0.2),
 (63, 0.4),
 (64, 0.6),
 (65, 0.4),
 (66, 0.4),
 (67, 0.4),
 (68, 0.4),
 (69, 0.4),
 (70, 0.2),
 (71, 0.4),
 (72, 0.4),
 (73, 0.6),
 (74, 0.6),
 (75, 0.6),
 (76, 0.4),
 (77, 0.0),
 (78, 0.6),
 (79, 0.8),
 (80, 0.4),
 (81, 0.6),
 (82, 0.0),
 (83, 0.8),
 (

In [415]:
test_x = gen_x(1, 3)
test_y = gen_y(test_x)
test_x[0,0], p.prediction(test_x), test_y

(23, 1.0, array([0.], dtype=float32))

In [ ]:
p = Perceptron(train_x, train_y, sigmoid, cross_entropy_deriv, cross_entropy_loss)
y_p = p.feed_forward()
y_p[:5]

TypeError: __init__() missing 1 required positional argument: 'loss'

In [ ]:
cross_entropy_deriv(y_p[:5], train_y[:5].flatten())
# # np.apply_along_axis(cross_entropy_deriv, 0, )
# # train_y[:5].T
# y_p.shape, train_y.shape
idx = np.random.randint(y_p.shape[0], size=3)
y_p

array([4, 4, 1])

In [ ]:
train_x[0].shape

(2,)